In [58]:
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras
from keras import layers
from keras import ops
import numpy as np
import string
import re
import warnings
warnings.filterwarnings("ignore")

text_file = "../assets/fin.txt"

with open(text_file, encoding='utf-8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, finnish, rest = line.split("\t")
    finnish = "[start] " + finnish + " [end]"
    text_pairs.append((english, finnish))    
    

In [31]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [32]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [33]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
            padding_mask = ops.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [34]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

In [35]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
 
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")
vocab_size = 15000
sequence_length = 20
 
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_finnish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_finnish_texts) 

In [36]:
batch_size = 64 
  
def format_dataset(eng, fin):
    eng = source_vectorization(eng)
    fin = target_vectorization(fin)
    return ({
        "english": eng,
        "finnish": fin[:, :-1],
    }, fin[:, 1:])
 
def make_dataset(pairs):
    eng_texts, fin_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    fin_texts = list(fin_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, fin_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()
 
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [41]:
embed_dim = 256 
dense_dim = 2048 
num_heads = 8 

vocab_size = 15000
sequence_length = 250
  
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
 
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="finnish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [42]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
791/791 ━━━━━━━━━━━━━━━━━━━━ 730s 914ms/step - accuracy: 0.1508 - loss: 5.0878 - val_accuracy: 0.1748 - val_loss: 3.3207
Epoch 2/30
791/791 ━━━━━━━━━━━━━━━━━━━━ 715s 904ms/step - accuracy: 0.1729 - loss: 3.4735 - val_accuracy: 0.1929 - val_loss: 2.8394
Epoch 3/30
791/791 ━━━━━━━━━━━━━━━━━━━━ 722s 912ms/step - accuracy: 0.1892 - loss: 2.9657 - val_accuracy: 0.1991 - val_loss: 2.6405
Epoch 4/30
791/791 ━━━━━━━━━━━━━━━━━━━━ 754s 953ms/step - accuracy: 0.1994 - loss: 2.6717 - val_accuracy: 0.2046 - val_loss: 2.5104
Epoch 5/30
791/791 ━━━━━━━━━━━━━━━━━━━━ 722s 912ms/step - accuracy: 0.2065 - loss: 2.4842 - val_accuracy: 0.2068 - val_loss: 2.4763
Epoch 6/30
791/791 ━━━━━━━━━━━━━━━━━━━━ 837s 1s/step - accuracy: 0.2120 - loss: 2.3615 - val_accuracy: 0.2078 - val_loss: 2.4845
Epoch 7/30
791/791 ━━━━━━━━━━━━━━━━━━━━ 858s 1s/step - accuracy: 0.2158 - loss: 2.2855 - val_accuracy: 0.2064 - val_loss: 2.5869
Epoch 8/30
791/791 ━━━━━━━━━━━━━━━━━━━━ 832s 1s/step - accuracy: 0.2186 - loss: 2.

In [64]:
def translate_sentence(input_sentence, generate_length=50, temperature=0.7):
    sentence = "[start] " + input_sentence
    tokens_index = dict(enumerate(target_vectorization.get_vocabulary()))
    
    for _ in range(generate_length):
        tokenized_sentence = target_vectorization([sentence])
        dummy_english_input = np.zeros((1, tokenized_sentence.shape[1]))
        
        predictions = transformer.predict(
            {"english": dummy_english_input, "finnish": tokenized_sentence}, verbose=0
        )
        
        next_token = sample_next(predictions[0, -1, :], temperature)
        sampled_token = tokens_index[next_token]
        
        if sampled_token == "[end]":
            break
        
        sentence += " " + sampled_token
    
    return sentence.replace("[start] ", "")

print(translate_sentence("moi mitä kuuluu"))

moi mitä kuuluu ollaan etsivän viivy salattavaa leivoin sinun oudolta jotka mariin pointti älykäs [UNK] sinun [UNK] kolumni [UNK] hillitä marialla ettet harvinaista lienee pelaamme samaa käske olkaa suolan puhelias tilanteen oma yläkoulussa suolaista vuosi kolmen ylennyksen pelkäsin syötävää asemaa optimistisempi olkaa koiranpentu aioin kadulla putosi vanha vietin käyntiin varanneet tarjoillaan silta osasi
